# Random Matrix Theory & Free Probability in Finance

Some random musings and a collection of ideas I hope to organize and push the idea that Random Matrix Theory (RMT) and Free Probability (FP) can be applied in the context of financial markets, specifically towards developing a free stochastic calculus and its applications. In addition to the perturbation of random matrices.


[//]: # "secret LaTeX commands"
$\renewcommand{\P}{\mathbb{P}}$
$\renewcommand{\F}{\mathcal{F}}$
$\renewcommand{\M}{\mathcal{M}}$
$\newcommand{\E}{\mathrm{E}}$

## Table of Contents
1. [Introduction](#introduction)
    1. [What Is a Random Matrix?](#whatisrm)
    2. [Where do we find random matrices?](#wheretofindrm)
2. [Example](#example)
3. [Example2](#example2)

## Introduction <a name ="introduction"></a>
A _Random Matrix_ is a matrix $A= (a_{ij})_{i,j=1}^N$ where the entries $a_{i,j} : \Omega \to \mathbb{C}$ are random variables on a probability space $(\Omega, \F, \P)$. Recall, that given a real-valued random variable $X:\Omega\to \R$, we may define its _distribution_ $F_X$ as $F_X(t) = \mathbb{P}(\{\omega\in \Omega : X(\omega) \leq t\})$. A complex random variable $Z$ can be defined as the sum of its real and imaginary parts, i.e. $Z = X + i Y$ with $X$ and $Y$ being real-valued random variables. If $X$ and $Y$ are independent then we can describe the joint distribution of $Z$ by the convolution of its real and imaginary parts, $F_Z = F_X \ast i F_Y$. 

Our main interest of study is the _eigenvalues_ of the random matrix. Recall that an _eigenvalue_ $\lambda \in \mathbb{C}$ of the $n\times n$ matrix $A$ is a is a number such that there exists a non-zero vector $x\in \R^n$ where $Ax = \lambda x$. In many applications, the matrices that crop up tend to be self-adjoint, i.e. $A^* = A$, where if $A$ has real-valued entries then its adjoint is equivalent to the transpose, and in general if it has complex-valued entries, its adjoint is taking the complex-conjugate of the transpose. For example, in quantum mechanics, physical observables like position, momentum and  energy are represented by self-adjoing operators where eigenvalues coresspond to the possible measured values of the observable. A correlation matrix too is self-adjoint, with its eigenvalues representing the amount of variance corresponding to each principal component. The wonderful thing about this is that the eigenvalues of a self-adjoint matrix are all real. 

### What is a Random Matrix? <a name ="whatisrm"></a>
here is what random matrix

### Where do we find random matrices?<a name="wheretofindrm"></a>
here's where we find them

#### Financial Data
Fix a probability space $(\Omega, \F, \P)$, here $\Omega$ is the set of all possible "wordls" (joint evolutions of all $N$ quantities across all $T$ sampling times). We have $N$ assets, observed at $T$ trading days. Here we can take $\Omega = \N$ or $\Omega = \R_{\geq 0}$. There are two equivalent but slgihtly different formalizations to keep in mind. 

__Time-series (process) viewpoint__

Define for each $i\in [N]$ a discrete-time stochastic process $\{R_i(t)\}_{i=1}^T$ where $R_i(t):\Omega \to \R$. For each fixed $t$, the $N$-vector $R(t) = (R_1(t),\ldots, R_N(t)):\Omega \to \R^N$ collects the $N$ quantities at sampling index $t$.

__Panel Data / Product-Space Viewpoint__

Equivalently, define a single $(T\times N)$-dimensional random array 
$$\mathcal{R}:\Omega \to \R^{T\times N} \qquad \mathcal{R}(\omega) = (R_i(t)(\omega))_{1\leq i\leq N}^{1\leq t \leq T}.$$
Then $R_i(t) = \pi_{t,i}\circ \mathcal{R}$ where $\pi_{t,i}$ projects onto the $(t,i)^\text{th}$ entry. 

__Population Correlation Matrix__

The true _population correlation matrix_ $C$ is a $N\times N$ matrix $C$ defined by 
$$C_{i,j} = \E(R_i(t)R_j(t)),$$
where if stationarity across $t$ is assumed then this matrix does not depend on $t$. Howevr, if time-stationarity fails, we may define $C_{i,j}$ by averaging over $t$ in some way. As the variables are standardized we have $C_{ii}=1$ and $C$ is symmetric positive semidefinite (positive definite unless there is a linear dependence almost surely). $C$ is theoretical and for most applications cannot be computed, however what we can compute from data is the _empirical estimator_ $E$. The whole field of random matrix noise-cleaning is about how $E$ deviates from $C$ for finite $T$ and large $N$. 

__Observed Data as the Realization__

Observe the return, this corresponds to reality unfolding and selecting a single $\omega^*\in \Omega$, then the random variable takes a numerical value:
$$r_{t,i} := R_i(t)(\omega^*)\in \R.$$
This is the _realization of the $i^\text{th}$ quantity at time $t$_ ,  the measured outcome in the data set. You never see $R_i(t)$ directly, you only see $r_{t,i}$ for the one $\omega^*$ that actually occured. Colect these into a $T\times N$ data matrix 
$$r = (r_{t,i})_{1\leq t \leq T}^{1\leq i \leq N}.$$ 

## Example2
some more text 2



In [ ]:
!pip install yfinance
!pip install pandas
!pip install numpy
!pip install matplotlib

For the following code, we will select $N$ many tickers, and then choose a time frame (start and end dates) and then the interval for our sampling. Then $T = (\mathtt{end\_ date}-\mathtt{start\_ date})/\mathtt{interval}$. Then we construct the $T\times N$ whose columns will be our tickers evolving over time.  

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import date, timedelta

def fetch_stock_data(ticker, start_date, end_date, interval='1d'):
    df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    return df

def fetch_and_process_data(tickers, start_date, end_date, interval='1d'):
    d0 = date.fromisoformat(start_date)
    d1 = date.fromisoformat(end_date)
    matrix = []
    price = []
    for ticker in tickers:
        df = fetch_stock_data(ticker, start_date, end_date, interval)
        if not df.empty:
            # Close to close log returns capture the full 24-hour cycle including the intraday return and overnight return. Better than open-to-open as that's more volatile.
            df['Return'] = np.log(df['Close'] / df['Close'].shift(1))
            # Drop the first row which will have NaN due to shift
            df = df.dropna()
            matrix.append(df['Return'].to_numpy())
            price.append(df['Close'].to_numpy())
    # Transpose to get T x N shape
    return np.column_stack(matrix), np.column_stack(price)

my_tickers = ["AAPL", "GOOGL", "MSFT", "SPY", "AMZN", "TSLA", "NFLX", "META", "NVDA", "BRK-B"]
start_date = "2023-01-01"
end_date = "2023-12-31"
interval = '1d'

data_matrix, price_matrix = fetch_and_process_data(my_tickers, start_date, end_date, interval)
print(price_matrix)
print(price_matrix.shape) 
print(data_matrix)
print(data_matrix.shape)  # Should print (T, N) 

import matplotlib.pyplot as plt
from matplotlib import gridspec

for i, ticker in enumerate(my_tickers):
    fig = plt.figure(figsize=(12, 7))

    # 2 rows, 1 column. The top plot (price) will be twice the height of the bottom one (returns).
    gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

    # --- Top Subplot for Price ---
    ax_price = plt.subplot(gs[0])
    ax_price.plot(price_matrix[:, i], color='tab:red', label=f'{ticker} Price')
    ax_price.set_ylabel('Price')
    ax_price.set_title(f'Price and Log Returns for {ticker}')
    ax_price.legend(loc='upper left')
    ax_price.grid(True, linestyle='--', alpha=0.6) # Add a grid for readability

    # --- Bottom Subplot for Log Returns ---
    # Share the x-axis with the price plot
    ax_return = plt.subplot(gs[1], sharex=ax_price)
    ax_return.plot(data_matrix[:, i], color='tab:blue', label=f'{ticker} Log Return')
    ax_return.set_ylabel('Log Return')
    ax_return.set_xlabel('Time')
    ax_return.legend(loc='upper left')
    ax_return.grid(True, linestyle='--', alpha=0.6)

    # Hide the x-tick labels on the top plot
    plt.setp(ax_price.get_xticklabels(), visible=False)

    # Visualize volatility from returns better by adjusting the y-axis limits
    ax_return.set_ylim(data_matrix[:, i].min() * 1.1, data_matrix[:, i].max() * 1.1)

    # Remove the vertical space between the subplots
    plt.subplots_adjust(hspace=0)

    plt.show()


__Normalized Data Matrix__

Now given the raw returns, we define a scaled version of the data matrix by normalizing each entry by the square root of the number of observations $\sqrt{T}$. This is done as the traditional mean-zero sample correlation estimator is given by $E=X^TX$ and this scaling bakes $1/T$ into the defintion of $X$. By centering, standardizing, and scaling by $1/\sqrt{T}$  we bake our information in, so let the new entries be given by $$r_{t,i}' = \frac{r_{t,i}- \mu_i}{s_i}$$ where $\mu_i$ is the sample mean, and $s_i$ is the Bessel correction variance. 

In [ ]:
# demean and scale the data matrix
# axis=0 means we are normalizing each column (each ticker)
# ddof=1 applies Bessel's correction for sample standard deviation
normalized_data_matrix = (data_matrix - np.mean(data_matrix, axis=0)) / np.std(data_matrix, axis=0, ddof=1)
print(normalized_data_matrix)
print(normalized_data_matrix.shape)  # Should still be (T, N)

for i, ticker in enumerate(my_tickers):
    plt.figure(figsize=(10, 5))
    plt.plot(normalized_data_matrix[:, i], label=f'{ticker} Normalized Log Return', color='tab:blue')
    plt.title(f'Normalized Log Returns for {ticker}')
    plt.xlabel('Time')
    plt.ylabel('Normalized Log Return')
    plt.axhline(0, color='black', linestyle='--', linewidth=0.8)  # Add a horizontal line at y=0
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.close()

__Producing the Normalized Data Matrix__ $X$  

We scale each of the normalized returns by $1/\sqrt{T}$ so that when we express our correlation estimator $E$ we back $1/T$ to prevent an extra prefactor. 

Indeed define $X_{t,i} = \frac{r_{t,i}}{\sqrt{T}}$ for $t\in [T]$ and $i\in [N]$. Then define the _empirical correlation matrix_ $\hat{E}$ by
$$\hat{E}(\omega) := \frac{1}{T}\sum_{t=1}^T R_i(t)(\omega)R_j(t)(\omega),$$
or equivalently
$$\hat{E}(\omega) = X(\omega)^T X(\omega).$$

After the world $\omega^*$ is realized and data is measured/observed, we evaluate this statistic at $\omega^*$ where we recall $R_i(t)(\omega) = r_{t,i}$. We drop the $\omega^*$ and retrieve a numeric $N\times N$ matrix 
$$E_{i,j} = \frac{1}{T}\sum_{t=1}^Tr_{i,t}r_{j,t}.$$

Thus it is important to think of $\hat{E}$ as a random matrix measurable with respect to $(\Omega, \F)$ and $E$ as its observed value at the realized outcome $\omega^*$. 

__ Relationship Between $E$ and $C$__

In expectation, $$\E(\hat{E}) = \frac{1}{T} \sum_{t=1}^T \E(R_i(t)R_j(t)) = C_{i,j}$$ if $\E(R_i(t)R_j(t))$ does not depend on $t$.

So the empirical estimator, under mild conditions, is unbiased for the population correlation matrix. But for finite $T$ especially when $N$ is of the same order as $T$, $\hat{E}$ will be noisy, and its eigenstructure can differ substantially from that of $C$. 

__ Random Matrix Model Layer__
In high-dimensional asymptotics, one often assumes the raw data matrix $R = (R_i(t))_{t,i}$ can be written as $$R = YC^{1/2}$$ where $Y$ has i.i.d standardized entries (mean 0, variance, 1, possibly heavier tails) and $C^{1/2}$ is a deterministic square-root of the true correlation matrix. Then 

$$\hat{E} = \frac{1}{T}R^TR = C^{1/2}\left(\frac{1}{T} Y^TY\right)C^{1/2}.$$


RMT provides a theoretical benchmark for the distribution of eigenvalues of a correlation matrix of purely random data. This is given by the Marchenko-Pastur distribution that provides a specific range $[\lambda_\text{min},\lambda_\text{max}]$ for the eigenvalues that is given by the arc-sine law. 

Yadda-yadda show proof for Marchenko-Pastur distribtuion, then calculate eigenvalues for our normalized, standardized dude, then see that eigenvectors represent portfolio of assets, and eigenvalues represent the variance or risk of that portfolio. Eigenvalues of empirical matrix that fall within this theoretical range are considered to be indistinguishable from random noise. The corressponding eigenvectors represent portfolios with no statistically significant, stable relationships. Meanwhile, the signal lies in eigenvalues who fall outside this range (typically above $\lambda_\text{max}$) are considered to contain genuine, non-random information. These larger eigenvalues often correspond to significant market-wide effects like the overall market movement or strong sector-specific correlations. 

## Appendix

### Definitions
#### Correlations
Let $X,Y$ be random variables with means $\mu_X, \mu_Y$ with __positive__ standard deviations $\sigma_X, \sigma_Y$ then the _correlation coefficient_ is given by $$\rho_{X,Y} = \frac{E((X-\mu_X)(Y-\mu_Y))}{\sigma_X\sigma_Y}$$

By Cauchy-Schwarz, $|\rho_{X,Y}| \leq 1$ with $+1$ representing perfect direct relationship, and $-1$ representing perfect inverse relationship. Any value on the interval $(-1,1)$ implies the degree of linear dependence. 

Given a series of $n$ measurements $\{(X_i,Y_i\}_{i=1}^n$ the _sample correlation coefficient_ can be used to estimate the population correlation $\rho_{X,Y}$ where the estimate becomes exact as $n\to \infty$. The sample correlation coefficient is defined as 
$$r_{xy} = \frac{1}{(n-1)s_xs_y}\sum_{i=1}^n (x_i-\overline{x})(y_i-\overline{y})$$

where $s_x,s_y$ are the corrected sample standard deviations$ and $\overline{x},\overline{y}$ are the sample means. 

Now suppose we have $n$ random variables $X_1, \ldots, X_n$, the $n\times n$ matrix $C = (c_{i,j})_{i,j}$ where each entry $c_{i,j} = \rho_{X_i,X_j} = \frac{\mathrm{cov}(X_i,X_j)}{\sigma_{X_i}\sigma_{X_j}}$ is called the _correlation matrix_. As the covariance of any random variable is simply the second moment, we have the diagonal elements are just 1. We also have that $c_{i,j}=c_{j,i}$ so the matrix is symmetric.

If the random variables are demeaned and standardized 
 
### Examples
#### Quantum Physics Observables are Self-Adjoint Operators
### Proofs
#### Joint Distribution of Independent RVs is given by convolution
#### Jointly Normal and Uncorrelated implies Independence
Let $X, Y$ be random variables who are jointly normal, i.e. $(X,Y)\sim \mathcal{N}_2((\E(X),\E(Y))^T,\Sigma)$ where $\Sigma = (\mathrm{Cov}(X_i,X_j))_{1\leq i,j\leq 2}$ and uncorrelated, i.e. $\rho_{X,Y}=0$ then $X$ and $Y$ are independent. 
#### Eigenvalues of Self-Adjoint Matrix are Real
Let $A$ be an $n\times n$ self-adjoint marix. Let $\lambda \in \mathbb{C}$ be an eigenvalue of $A$ and $x\in \R^n$ its corresponding eigenvector. Then by the properties of the inner product we have $$\langle Ax , x\rangle = \langle x, A^* x\rangle = \langle x, Ax\rangle.$$ But as $\lambda$ is an eignevalue we have that $Ax=\lambda x$. Thus $$\lambda ||x||^2 = \langle \lambda x, x\rangle = \langle x, \lambda x \rangle = \overline{\langle \lambda x, x \rangle} = \overline{\lambda} ||x||^2.$$ As $x\neq 0$ then $||x||\neq 0$ hence we have $\lambda = \overline{\lambda}$ and thus $\lambda$ must be real-valued.
